In [1]:
#from vm_analysis import PEAnalysis, Mnemonic, WProtectEmulator
from miasm2.core.utils import upck32
import yara
import Levenshtein
from jinja2 import Template

In [2]:
from vm.mnemonic import WProtectMnemonic
from vm.analysis import PEAnalysis, WProtectEmulator
from vm.obfuscation import get_bytes, strip_vm_obfuscation
from vm.rule_generator import generate_rules

In [5]:
%%capture

import os

dataz = map(lambda x: WProtectEmulator("train_batch/" + x), filter(lambda x: "inceax.wp" in x, os.listdir("train_batch/")))
map(lambda x: x.find(), dataz)
dataz = map(lambda x: x.recover_mnemonics(x.offset, WProtectMnemonic)[1], dataz)
center = WProtectEmulator("helloWProtect.wp.exe")
center.find()
center = center.recover_mnemonics(center.offset, WProtectMnemonic)[1]

[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	PE loaded
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a99d
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a96e
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a93e
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a98c
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a942
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a9b2
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a9d5
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a97c
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40a961
[INFO]	reachable cod

In [5]:
def match_mnemonics(mnemonic, mnemonic_sets):
    def hexify(bytecode):
        return "".join(map(lambda x: "%.2X" % ord(x), list("".join(bytecode))))
    matches = []
    for mn_set in mnemonic_sets:
        distances = map(lambda x: Levenshtein.distance(hexify(get_bytes(strip_vm_obfuscation(mnemonic.code))),
                                                       hexify(get_bytes(strip_vm_obfuscation(x.code)))),
                        mn_set)
        matches.append(mn_set[distances.index(min(distances))])
    return matches

batches = []
for mnemonic in center:
    batches.append(match_mnemonics(mnemonic, dataz))

In [6]:
generate_rules(center)

In [3]:
center = WProtectEmulator("helloWProtect.wp.exe")
center.find()
center = center.recover_mnemonics(center.offset, WProtectMnemonic)[1]

[INFO]	PE loaded
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40b9d1
[INFO]	Mnemonics loaded


In [4]:
WProtectMnemonic.consolidate_names(center)

In [5]:
map(lambda x: x.name, center)

['mnemonic_0',
 'mnemonic_1',
 'mnemonic_2',
 'mnemonic_3',
 'mnemonic_4',
 'mnemonic_5',
 'mnemonic_6',
 'mnemonic_7',
 'mnemonic_8',
 'mnemonic_9',
 'mnemonic_10',
 'mnemonic_11',
 'mnemonic_12',
 'mnemonic_13',
 'mnemonic_14',
 'mnemonic_15',
 'mnemonic_16',
 'mnemonic_17',
 'mnemonic_18',
 'mnemonic_19',
 'mnemonic_20',
 'mnemonic_21',
 'mnemonic_22',
 'mnemonic_23',
 'mnemonic_24',
 'mnemonic_25',
 'mnemonic_26',
 'mnemonic_27',
 'mnemonic_28',
 'mnemonic_29',
 'mnemonic_30',
 'mnemonic_31',
 'mnemonic_32',
 'nop',
 'mnemonic_34',
 'mnemonic_35',
 'mnemonic_36',
 'mnemonic_37',
 'mnemonic_38',
 'mnemonic_39',
 'mnemonic_40',
 'mnemonic_41',
 'mnemonic_42',
 'mnemonic_43',
 'mnemonic_44',
 'mnemonic_45',
 'mnemonic_46',
 'mnemonic_47',
 'mnemonic_48',
 'mnemonic_49',
 'mnemonic_50',
 'mnemonic_51',
 'mnemonic_52',
 'mnemonic_53',
 'mnemonic_54',
 'mnemonic_55']

In [14]:
[map(lambda y: hex(ord(y)), list(x.b)) for x in center[53].code]

[['0x66', '0x43'],
 ['0xc1', '0xc3', '0xd'],
 ['0x66', '0xf7', '0xd3'],
 ['0xf7', '0xd3'],
 ['0x66', '0xf7', '0xd3'],
 ['0x66', '0xc1', '0xc3', '0x1b'],
 ['0x4b'],
 ['0x66', '0x8b', '0x46', '0xfe'],
 ['0x83', '0xee', '0x2'],
 ['0x66', '0x5', '0x1d', '0x39'],
 ['0x66', '0xc1', '0xc8', '0x15'],
 ['0x66', '0x40'],
 ['0x66', '0x2b', '0xc3'],
 ['0x66', '0xc1', '0xc0', '0x10'],
 ['0x66', '0xc1', '0xc0', '0x17'],
 ['0x66', '0x48'],
 ['0xf', '0xbf', '0xc0'],
 ['0x83', '0xed', '0x4'],
 ['0x89', '0x45', '0x0'],
 ['0xe9', '0xda', '0xfa', '0xff', '0xff']]

In [15]:
ruleset = yara.compile("mnemonics.yara")

In [16]:
ruleset.match(data="".join([x.b for x in center[53].code]))

[nop, mnemonic_53]